In [ ]:
! pip install googletrans

In [ ]:
import json

from googletrans import Translator

async def translate_markdown_in_ipynb(input_filename, output_filename):
    # .ipynbファイルを読み込む
    with open(input_filename, "r", encoding="utf-8") as f:
        notebook_json = json.load(f)

    async with Translator() as translator:
        # 各セルをチェックし、Markdownセルであれば翻訳
        for cell in notebook_json.get("cells", []):
            if cell.get("cell_type") == "markdown":
                # セル内のテキストは通常リスト形式なので、結合して一つの文字列にする
                original_text = "".join(cell.get("source", []))
                if original_text.strip():  # 空文字列でなければ
                    translated = await translator.translate(original_text, dest="ja")
                    translated_text = translated.text
                    # 翻訳結果をセルの内容に置換（リスト形式で保存）
                    cell["source"] = [translated_text]

    # 翻訳後の内容を保存
    with open(output_filename, "w", encoding="utf-8") as f:
        json.dump(notebook_json, f, ensure_ascii=False, indent=2)

In [ ]:
input_filename = input("ファイル名").strip()
assert input_filename.endswith(".ipynb"), "ファイル名は.ipynb形式である必要があります。"

output_filename = input_filename.replace(
    ".ipynb", "-ja.ipynb"
)  # 拡張子の前に "-ja" をつける

In [ ]:
await translate_markdown_in_ipynb(input_filename, output_filename)